In [1]:
#read all files into an array (dataframes)
import numpy as np
import os
import pandas as pd
from IPython.display import display
import datetime
import re
import json
directory = os.getcwd() 

def read_data_from_files(directory):
  dataframes= []
  for root,dirs,files in os.walk(directory):
      for file in files:
         if (file.endswith("Buildingslogs.csv") & os.path.exists(file)):
            col_names = pd.read_csv(file, nrows=0).columns

            dataframes.append(pd.read_csv(file,sep=";",low_memory=False,header=[0],skiprows=[1]))
  return dataframes


#group the data by building where d[str(serial)] = group

def group_data_by_building(data):
  d = {}
  for data in data :
    for serial, group in data.groupby('serial'):
      d[str(serial)] = group
  return d


def int_to_time(input,hours_mapping):
  return datetime.datetime(2021, 1, 1) + datetime.timedelta(hours=input/hours_mapping)

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +datetime.timedelta(hours=t.minute//30))  

def map_to_real_time(data):
  # hou much is an hour in game time 
  # for example 1hour = 2000 game time unite
  hour_in_numbers = (int(data[0]["time"].max()) - int(data[0]["time"].min())) / (365*24)
  for df in data :
    df["time"]=df["time"].apply(int_to_time,args=[hour_in_numbers])
    df["time"]=df["time"].apply(hour_rounder)
  return data  

def read_orders_data_from_files(directory):
  dataframes= []
  for root,dirs,files in os.walk(directory):
      for file in files:
         if (file.endswith("orderslogs.csv") & os.path.exists(file)):
            col_names = pd.read_csv(file, nrows=0).columns

            dataframes.append(pd.read_csv(file,sep=";",low_memory=False,header=[0],skiprows=[1]))
  return dataframes

#get one buildings stockes according to IBP Stock
#data_line is one line where wares != nan of the original data
def data_line_to_one_building_stock_IBP(position):
  #locations = pd.DataFrame(columns=['LOCNO','LOCTYPE','NAME1','NAME2','LAND1','REGIO','ORT01','PSTLZ','PSTL2','STRAS','LOEVM','TIME_ZONE','TELF1','FABKL','KUKLA','NODETYPE','CUST_ATTR1','CUST_ATTR2','CUST_ATTR3','CUST_ATTR4','CUST_ATTR5','CUST_ATTR6','CUST_ATTR7','CUST_ATTR8','CUST_ATTR9','CUST_ATTR10','LONGITUDE','LATITUDE'])
  #stocks = pd.DataFrame(columns=['MATNR','LOC_WERKS','VRFKZ','MNG01'])
  locatins=["0"]*4
  locatins[0]=position
  locatins[1]="V"
  locatins[2]="CET"
  locatins[3]="03"
  #locations["LOCNO"]=position
  #locations["LOCTYPE"]="V"
  #locations["TIME_ZONE"]="CET"
  #locations["FABKL"]="03"
  return locatins
  #group_data_by_building

def write_series_of_location_to_csv_as_IBPlocation(locationseries,filename):

  IBPlocations = pd.DataFrame(columns=['MANDT','LOGSYS','LOCNO','LOCTYPE','NAME1','NAME2','NAME3','NAME4','LAND1','REGIO','ORT01','PSTLZ','PSTL2','STRAS','LOEVM','TIME_ZONE','TELF1','FABKL','KUKLA','NODETYPE','CUST_ATTR1','CUST_ATTR2','CUST_ATTR3','CUST_ATTR4','CUST_ATTR5','CUST_ATTR6','CUST_ATTR7','CUST_ATTR8','CUST_ATTR9','CUST_ATTR10','LONGITUDE','LATITUDE'])
  IBPlocations["LOCNO"]=locationseries
  #x = set([x[0] for x in test["position"]])
  #locations["'LATITUDE'"]=list(x)
  IBPlocations["MANDT"]="001"
  IBPlocations["LOGSYS"]="WDL"
  IBPlocations["LOCTYPE"]="P"
  IBPlocations["TIME_ZONE"]="CET"
  IBPlocations["FABKL"]="03"
  IBPlocations.to_csv("locations"+filename+ ".csv",sep=";",index=False,header=False)

def write_singel_string_wares_attribute_to_csv_as_IBPmaterial(wares):
  IBPmaterial = pd.DataFrame(columns=['MATNR','LVORM','MATKL','MEINS','PRDHA','MTART','CUST_ATTR1','CUST_ATTR2','CUST_ATTR3','CUST_ATTR4','CUST_ATTR5','CUST_ATTR6','CUST_ATTR7','CUST_ATTR8','CUST_ATTR9','CUST_ATTR10'])
  wares = re.sub(r".*wares\:", "", wares)
  Dict = json.loads(wares)
  wares_amount_data = pd.DataFrame.from_dict(Dict.items())
  wares_amount_data.columns = ['ware', 'amount']
  IBPmaterial["MATNR"]=wares_amount_data["ware"]
  IBPmaterial["MEINS"]="piece"
  IBPmaterial.to_csv("materials.csv",sep=";",index=False,header=False)

def write_one_buildings_entry_to_IBPstock(building_entry,filename):
  IBPstocks = pd.DataFrame(columns=['MATNR','LOC_WERKS','VRFKZ','MNG01'])
  print(building_entry["serial"])
  
  wares=building_entry["wares"]
  wares = re.sub(r".*wares\:", "", wares)
  Dict = json.loads(wares)
  wares_amount_data = pd.DataFrame.from_dict(Dict.items())
  wares_amount_data.columns = ['ware', 'amount']
  IBPstocks["MATNR"]=wares_amount_data["ware"]
  IBPstocks["MNG01"]=wares_amount_data["amount"]
  IBPstocks["VRFKZ"]="X"
  IBPstocks['LOC_WERKS']=str(building_entry["serial"])
  IBPstocks.to_csv(filename+"stocks.csv",sep=";",index=False,header=False)



  


In [2]:
data = read_data_from_files(directory)


In [3]:
a=map_to_real_time(read_data_from_files(directory))


In [4]:
string=a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ]["wares"][0]
write_singel_string_wares_attribute_to_csv_as_IBPmaterial(a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ]["wares"][0])

In [5]:
write_one_buildings_entry_to_IBPstock(a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ].iloc[0],str(a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ].iloc[0]["serial"]))

1461


In [6]:
string=a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ]["wares"][0]
string = re.sub(r".*wares\:", "", string)

In [7]:
Dict = json.loads(string)
fuck = pd.DataFrame.from_dict(Dict.items())
fuck.columns = ['ware', 'amount']

In [8]:
orders=read_orders_data_from_files(directory)

In [9]:
orders_maped = map_to_real_time(orders)
orders[0].rename(columns={'building_serial':'serial'}, inplace=True)
merged_data = a[0].merge(orders[0], on=["serial","time"], how='left')
print(merged_data[~merged_data["requested_Ware"].isnull()])

        serial                 name_x                time  \
2935      4406  atlanteans_spiderfarm 2021-01-22 03:00:00   
2936      4406  atlanteans_spiderfarm 2021-01-22 03:00:00   
2937      4406  atlanteans_spiderfarm 2021-01-22 03:00:00   
2938      4406  atlanteans_spiderfarm 2021-01-22 03:00:00   
2939      4406  atlanteans_spiderfarm 2021-01-22 03:00:00   
...        ...                    ...                 ...   
385365    6554        atlanteans_mill 2021-12-31 22:00:00   
385366    6554        atlanteans_mill 2021-12-31 22:00:00   
385367    6554        atlanteans_mill 2021-12-31 22:00:00   
385369    6554        atlanteans_mill 2021-12-31 22:00:00   
385370    6554        atlanteans_mill 2021-12-31 22:00:00   

        ware_economy_serial  worker_economy_serial position  \
2935                     16                     17  (182,9)   
2936                     16                     17  (182,9)   
2937                     16                     17  (182,9)   
2938           

In [10]:
for i in range(len(data)):
  write_series_of_location_to_csv_as_IBPlocation(data[i]["serial"].unique(),str(i))


